# Checking vs. trying all possible cases

In [1]:
%matplotlib inline
def documents_ok(passport,permanent_resident_card,drivers_license,voter_registration,under_18,report_card,doctor_record,daycare_record,school_id,social_security_card,birth_certificate):
    return (passport or permanent_resident_card) or \
            ((drivers_license or school_id or voter_registration) or \
                (under_18 and \
                (report_card or doctor_record or daycare_record)) and \
            (social_security_card or birth_certificate))


import itertools
works=[]
doesnt_work=[]
for combo in itertools.product([True,False],repeat=11):
    if documents_ok(combo[0],combo[1],combo[2],combo[3],combo[4],combo[5],combo[6],combo[7],combo[8],combo[9],combo[10]):
        works+=[combo]
    else:
        doesnt_work+=[combo]


# 3SAT

In [2]:
def checker(f,candidate):
    """
    f: can be any function which takes as a string of 1s and 0s which is at least as long as the number of variables in f. 
    Right most character is the 0th bit.
    candidate: should be a string of 1s and 0s which is at least as long as the number of variables in f. 
    Right most character is the 0th bit.
    """
    return(f(candidate))

def try_all_inputs(f,len_input):
    import itertools
    result=[]
    for candidate in ["".join(seq) for seq in itertools.product("01", repeat=len_input)]:
        if checker(f,candidate):
            result+=[candidate]
    return result

def is_satisfiable(f,len_input):
    return len(try_all_inputs(f,len_input))>0
            

def a_3sat_function_4(binary_string):
    """
    binary_string is a string that is at least 4 characters long with 1s and 0s with the 
    rightmost character representing the 0th bit
    """
    binary_list=[int(i) for i in binary_string]
    binary_list.reverse()
    a,b,c,d=binary_list[0:4]
    return (not a or b or d) and \
            (not b or c or d) and \
            (a or not c or d) and \
            (not a or not b or not d) and \
            (b or not c or not d) and \
            (not a or c or not d) and \
            (a or b or c) and \
            (not a or not b or not c) 

print(is_satisfiable(a_3sat_function_4,4))
print(try_all_inputs(a_3sat_function_4,4))


True
['1010', '1110']


In [4]:
def a_mystery_function_3(binary_string):
    """
    binary_string is a string that is at least 4 characters long with 1s and 0s with the 
    rightmost character representing the 0th bit
    """
    binary_list=[int(i) for i in binary_string]
    binary_list.reverse()
    a,b,c=binary_list[0:3]
    return (a or b or not c) and \
            (a or b or c) and \
            (a or not b or c) and \
            (a or not b or not c) and \
            (not a or b or not c) and \
            (not a or b or c) and \
            (not a or not b or not c)


def a_mystery_function_2(binary_string):
    """
    binary_string is a string that is at least 4 characters long with 1s and 0s with the 
    rightmost character representing the 0th bit
    """
    binary_list=[int(i) for i in binary_string]
    binary_list.reverse()
    a,b=binary_list[0:2]
    return a and not b

In [5]:
print(try_all_inputs(a_mystery_function_3,3))

print(try_all_inputs(a_mystery_function_2,2))


['011']
['01']


# Toffoli Gate

In [6]:
from qiskit.extensions.standard import ccx

<ipython-input-6-a3f4cf5c07ad>:1: DeprecationWarning: The module qiskit.extensions.standard is deprecated as of 0.14.0 and will be removed no earlier than 3 months after the release. You should import the standard gates from qiskit.circuit.library.standard_gates instead.
  from qiskit.extensions.standard import ccx
<ipython-input-6-a3f4cf5c07ad>:1: DeprecationWarning: This module is deprecated.
  from qiskit.extensions.standard import ccx


In [7]:
a=True
b=False
if a and b:
    print("Ready for the next section")
else:
    print("Change one variable to move on")

Change one variable to move on


# Running all combinations of inputs in the Toffoli full gate specification 

In [8]:
import qiskit
from qiskit import Aer
from qiskit import QuantumCircuit
qasm_header="""
include "qelib1.inc";
qreg q[3];
creg c[3];
"""

qasm_toffoli="""
h q[2];
cx q[1],q[2]; 
tdg q[2]; 
cx q[0],q[2];
t q[2];
cx q[1],q[2];
tdg q[2];
cx q[0],q[2];
t q[1];
t q[2];
h q[2];
cx q[0],q[1];
t q[0];
tdg q[1];
cx q[0],q[1];
measure q[0] -> c[0];
measure q[1] -> c[1];
measure q[2] -> c[2];
"""

print("inputs","outputs")
print("abc","a'b'c'")
for abc in [(0,0,0),
                (0,0,1),
                (0,1,0),
                (0,1,1),
                (1,0,0),
                (1,0,1),
                (1,1,0),
                (1,1,1)]:

    qasm_input=""
    for i in range(3):
        if abc[i]:
            qasm_input+="""x q[%d];
            """%i
    qasm_string=qasm_header+qasm_input+qasm_toffoli

    qc = QuantumCircuit.from_qasm_str(qasm_string)
    # Run on local simulator
    backend = Aer.get_backend('qasm_simulator')
    sim = qiskit.execute(qc,backend=backend)
    result = sim.result()
    # Output result
    print("".join([str(i) for i in abc]),list(result.get_counts(qc).keys())[0][::-1])
    

inputs outputs
abc a'b'c'
000 000
001 001
010 010
011 011
100 100
101 101
110 111
111 110


# Using QISKit's ccx gate instead of full specification

In [9]:
import qiskit
from qiskit import Aer
from qiskit import ClassicalRegister, QuantumRegister
from qiskit.extensions.standard import ccx
from qiskit import QuantumCircuit, execute

print("inputs","outputs")
print("abc","a'b'c'")
for abc in [(0,0,0),
                (0,0,1),
                (0,1,0),
                (0,1,1),
                (1,0,0),
                (1,0,1),
                (1,1,0),
                (1,1,1)]:

    # set up registers and program
    qr = QuantumRegister(3)
    cr = ClassicalRegister(3)
    qc = QuantumCircuit(qr, cr)

    for i in range(3):
        if abc[i]:
            qc.x(qr[i])
    qc.ccx(qr[0],qr[1],qr[2])
    for i in range(3):
        qc.measure(qr[i], cr[i])

    # Run on local simulator
    backend = Aer.get_backend('qasm_simulator')
    sim = qiskit.execute(qc,backend=backend)
    result = sim.result()
    # Output result
    print("".join([str(i) for i in abc]),list(result.get_counts(qc).keys())[0][::-1])




inputs outputs
abc a'b'c'
000 000
001 001
010 010
011 011
100 100
101 101
110 111
111 110


# Showing the Quantum OR gate truth table

In [10]:
import qiskit
from qiskit import Aer
from qiskit import QuantumCircuit
qasm_header="""
include "qelib1.inc";
qreg q[3];
creg c[3];
"""

qasm_quantumor="""
x q[0];
x q[1];
x q[2];
h q[2];
cx q[1],q[2]; 
tdg q[2]; 
cx q[0],q[2];
t q[2];
cx q[1],q[2];
tdg q[2];
cx q[0],q[2];
t q[1];
t q[2];
h q[2];
cx q[0],q[1];
t q[0];
tdg q[1];
cx q[0],q[1];
x q[0];
x q[1];
measure q[0] -> c[0];
measure q[1] -> c[1];
measure q[2] -> c[2];
"""

print("inputs","outputs")
print("abc","a'b'c'")
for abc in [(0,0,0),
                (0,0,1),
                (0,1,0),
                (0,1,1),
                (1,0,0),
                (1,0,1),
                (1,1,0),
                (1,1,1)]:

    qasm_input=""
    for i in range(3):
        if abc[i]:
            qasm_input+="""x q[%d];
            """%i
    qasm_string=qasm_header+qasm_input+qasm_quantumor
    qc = QuantumCircuit.from_qasm_str(qasm_string)
    # Run on local simulator
    backend = Aer.get_backend('qasm_simulator')
    sim = qiskit.execute(qc,backend=backend)
    result = sim.result()
    # Output result
    print("".join([str(i) for i in abc]),list(result.get_counts(qc).keys())[0][::-1])

inputs outputs
abc a'b'c'
000 000
001 001
010 011
011 010
100 101
101 100
110 111
111 110


# Shortest

In [2]:
import numpy as np
from qiskit import BasicAer
from qiskit.aqua import QuantumInstance#, run_algorithm
from qiskit.aqua.algorithms import Grover
from qiskit.aqua.components.oracles import LogicalExpressionOracle, TruthTableOracle

#3-SAT Oracle in text format
input_3sat = '''
c example DIMACS-CNF 3-SAT
p cnf 3 5
-1 -2 -3 0
1 -2 3 0
1 2 -3 0
1 -2 -3 0
-1 2 3 0
'''

#Create a logical expression oracle based on the textual description
oracle = LogicalExpressionOracle(input_3sat)
#Use the oracle in Grover's Algorithm
grover = Grover(oracle)

backend = BasicAer.get_backend('qasm_simulator')
quantum_instance = QuantumInstance(backend, shots=1024)
#Run Grover's Algorithm in a Quantum Instance
result = grover.run(quantum_instance)
print(result['measurement'])


{'010': 26, '111': 36, '110': 30, '000': 292, '001': 24, '101': 288, '011': 293, '100': 35}
